# Loan Payment Prediction
Data set description
Question: Will the borrower pay the loan? - Prediction

Columns to take a look at:
- Columns that leak info from the future -to be avoided in our model
- Columns that don't impact the borrower's ability to pay loan
- Columns that need data cleaning
- Missing data
- Combination of columns to get features
- Redundant columns

Which is our target column to predict the loan payment? `loan_status`

In [29]:
# Disable warnings in Anaconda
import warnings
warnings.filterwarnings('ignore')

import pandas as pd # Data processing
pd.options.display.max_columns = 52

import numpy as np # Linear algebra

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_palette("colorblind")

In [2]:
loans_2007 = pd.read_csv('loans_2007.csv')

In [3]:
loans_2007.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
0,1077501,1296599.0,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,NaN,10+ years,RENT,24000.0,Verified,Dec-2011,Fully Paid,n,credit_card,Computer,860xx,AZ,27.65,0.0,Jan-1985,1.0,3.0,0.0,13648.0,83.7%,9.0,f,0.00,0.00,5863.155187,5833.84,5000.00,863.16,0.00,0.00,0.00,Jan-2015,171.62,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
1,1077430,1314167.0,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,Ryder,< 1 year,RENT,30000.0,Source Verified,Dec-2011,Charged Off,n,car,bike,309xx,GA,1.00,0.0,Apr-1999,5.0,3.0,0.0,1687.0,9.4%,4.0,f,0.00,0.00,1008.710000,1008.71,456.46,435.17,0.00,117.08,1.11,Apr-2013,119.66,Sep-2013,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
2,1077175,1313524.0,2400.0,2400.0,2400.0,36 months,15.96%,84.33,C,C5,NaN,10+ years,RENT,12252.0,Not Verified,Dec-2011,Fully Paid,n,small_business,real estate business,606xx,IL,8.72,0.0,Nov-2001,2.0,2.0,0.0,2956.0,98.5%,10.0,f,0.00,0.00,3005.666844,3005.67,2400.00,605.67,0.00,0.00,0.00,Jun-2014,649.91,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
3,1076863,1277178.0,10000.0,10000.0,10000.0,36 months,13.49%,339.31,C,C1,AIR RESOURCES BOARD,10+ years,RENT,49200.0,Source Verified,Dec-2011,Fully Paid,n,other,personel,917xx,CA,20.00,0.0,Feb-1996,1.0,10.0,0.0,5598.0,21%,37.0,f,0.00,0.00,12231.890000,12231.89,10000.00,2214.92,16.97,0.00,0.00,Jan-2015,357.48,Apr-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
4,1075358,1311748.0,3000.0,3000.0,3000.0,60 months,12.69%,67.79,B,B5,University Medical Group,1 year,RENT,80000.0,Source Verified,Dec-2011,Current,n,other,Personal,972xx,OR,17.94,0.0,Jan-1996,0.0,15.0,0.0,27783.0,53.9%,38.0,f,461.73,461.73,3581.120000,3581.12,2538.27,1042.85,0.00,0.00,0.00,Jun-2016,67.79,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0


In [4]:
loans_2007.shape

(42538, 52)

In [5]:
relevant_columns = ['loan_amnt','term','int_rate','installment',
                   'emp_length','home_ownership','annual_inc',
                   'pymnt_plan','purpose','addr_state','delinq_2yrs',
                   'earliest_cr_line','inq_last_6mths',
                    'open_acc','pub_rec','total_acc','application_type',
                    'tax_liens','verification_status','title','dti',
                    'revol_bal','revol_util','last_credit_pull_d', 
                    'pub_rec_bankruptcies',                 
                    'loan_status' #target col
                   ]
# grade and sub_grade not included since they include redundant info from int_rate
# earliest_cr_line : create new feature: years since first credit line
#inq_last_ [6 and 12] months: figure sth out
relevant_columns
drop_columns = loans_2007.columns.difference(relevant_columns)
drop_columns

Index(['acc_now_delinq', 'chargeoff_within_12_mths', 'collection_recovery_fee',
       'collections_12_mths_ex_med', 'delinq_amnt', 'emp_title', 'funded_amnt',
       'funded_amnt_inv', 'grade', 'id', 'initial_list_status', 'issue_d',
       'last_pymnt_amnt', 'last_pymnt_d', 'member_id', 'out_prncp',
       'out_prncp_inv', 'policy_code', 'recoveries', 'sub_grade',
       'total_pymnt', 'total_pymnt_inv', 'total_rec_int', 'total_rec_late_fee',
       'total_rec_prncp', 'zip_code'],
      dtype='object')

In [6]:
loans_2007=loans_2007.drop(drop_columns,axis=1) #drop columns

In [7]:
loans_2007['loan_status'].value_counts()

Fully Paid                                             33136
Charged Off                                             5634
Does not meet the credit policy. Status:Fully Paid      1988
Current                                                  961
Does not meet the credit policy. Status:Charged Off      761
Late (31-120 days)                                        24
In Grace Period                                           20
Late (16-30 days)                                          8
Default                                                    3
Name: loan_status, dtype: int64

In [8]:
# keep loans fully paid or charged off only
loans_2007 = loans_2007[
    (loans_2007['loan_status'] == 'Fully Paid') |
    (loans_2007['loan_status'] == 'Charged Off') 
     ]

# transform fully paid and charged off to numeric values
mapping_dict = {
    'loan_status': {
        'Fully Paid': 1,
        'Charged Off': 0
    }
}
loans_2007 = loans_2007.replace(mapping_dict)

In [9]:
loans_2007['loan_status'].value_counts() # confirm changes

1    33136
0     5634
Name: loan_status, dtype: int64

There are way more paid off than charged off loans, which means that our model will tend to predict more 1 (paid off) loans than charged off - biased model.

In [10]:
loans_2007['tax_liens'].unique()

array([ 0., nan])

In [11]:
# drop columns that contain only 1 unique value besides null

drop_columns = []

for col in loans_2007.columns:
    non_null_vals = loans_2007[col].dropna()
    unique_vals = non_null_vals.unique()
    num_unique_vals = len(unique_vals)
    if num_unique_vals == 1:
        drop_columns.append(col)

loans_2007 = loans_2007.drop(drop_columns,axis=1)
drop_columns # to see which columns were dropped
loans_2007.head()

,loan_amnt,term,int_rate,installment,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,title,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,last_credit_pull_d,pub_rec_bankruptcies
0,5000.0,36 months,10.65%,162.87,10+ years,RENT,24000.0,Verified,1,credit_card,Computer,AZ,27.65,0.0,Jan-1985,1.0,3.0,0.0,13648.0,83.7%,9.0,Jun-2016,0.0
1,2500.0,60 months,15.27%,59.83,< 1 year,RENT,30000.0,Source Verified,0,car,bike,GA,1.00,0.0,Apr-1999,5.0,3.0,0.0,1687.0,9.4%,4.0,Sep-2013,0.0
2,2400.0,36 months,15.96%,84.33,10+ years,RENT,12252.0,Not Verified,1,small_business,real estate business,IL,8.72,0.0,Nov-2001,2.0,2.0,0.0,2956.0,98.5%,10.0,Jun-2016,0.0
3,10000.0,36 months,13.49%,339.31,10+ years,RENT,49200.0,Source Verified,1,other,personel,CA,20.00,0.0,Feb-1996,1.0,10.0,0.0,5598.0,21%,37.0,Apr-2016,0.0
5,5000.0,36 months,7.90%,156.46,3 years,RENT,36000.0,Source Verified,1,wedding,My wedding loan I promise to pay back,AZ,11.20,0.0,Nov-2004,3.0,9.0,0.0,7963.0,28.3%,12.0,Jan-2016,0.0


In [12]:
null_vals = loans_2007.isnull().sum()
null_vals[null_vals > 0]

emp_length              1036
title                     11
revol_util                50
last_credit_pull_d         2
pub_rec_bankruptcies     697
dtype: int64

In [13]:
loans_2007['pub_rec_bankruptcies'].value_counts()

0.0    36422
1.0     1646
2.0        5
Name: pub_rec_bankruptcies, dtype: int64

As we can see above, `pub_rec_bankruptcies` offers little to no variability. We'll drop this column.

In [14]:
loans_2007.drop(['pub_rec_bankruptcies'],axis=1,inplace=True)

In [15]:
print('Missing emp_length values: {:.2f}%'.format(
      100 * null_vals['emp_length']/len(loans_2007)))

Missing emp_length values: 2.67%


Although we could impute the missing `emp_length` values using the mode (10+ years), we can afford dropping all the rows with na values, since they represent less than a 3% of the total number of rows.

In [16]:
loans_2007.dropna(inplace=True)
loans_2007 = loans_2007.reset_index(drop=True) # reset indexes
loans_2007.isnull().sum()

loan_amnt              0
term                   0
int_rate               0
installment            0
emp_length             0
home_ownership         0
annual_inc             0
verification_status    0
loan_status            0
purpose                0
title                  0
addr_state             0
dti                    0
delinq_2yrs            0
earliest_cr_line       0
inq_last_6mths         0
open_acc               0
pub_rec                0
revol_bal              0
revol_util             0
total_acc              0
last_credit_pull_d     0
dtype: int64

## Handling text columns

In [17]:
loans_2007.dtypes.value_counts()

object     11
float64    10
int64       1
dtype: int64

In [18]:
object_cols_df = loans_2007.select_dtypes(include='object')
object_cols_df.head(1)

,term,int_rate,emp_length,home_ownership,verification_status,purpose,title,addr_state,earliest_cr_line,revol_util,last_credit_pull_d
0,36 months,10.65%,10+ years,RENT,Verified,credit_card,Computer,AZ,Jan-1985,83.7%,Jun-2016


In [19]:
drop_cols = ['last_credit_pull_d', #requires exhaustive feature engineering
             'addr_state', #would require 49 dummy variables
             'title', #too many values
             'earliest_cr_line' #requires exhaustive feature engineering
            ]
loans_2007.drop(drop_cols,axis=1,inplace=True)

loans_2007['int_rate']=loans_2007['int_rate'].str.rstrip('%')
loans_2007['int_rate']=loans_2007['int_rate'].astype('float')

loans_2007['revol_util']=loans_2007['revol_util'].str.rstrip('%')
loans_2007['revol_util']=loans_2007['revol_util'].astype('float')

mapping_emp_length = { #converting to numeric values instead of categories
    "emp_length": {
        "10+ years": 10,
        "9 years": 9,
        "8 years": 8,
        "7 years": 7,
        "6 years": 6,
        "5 years": 5,
        "4 years": 4,
        "3 years": 3,
        "2 years": 2,
        "1 year": 1,
        "< 1 year": 0,
        "n/a": 0
    }
}

loans_2007=loans_2007.replace(mapping_emp_length)

loans_2007.head()

,loan_amnt,term,int_rate,installment,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc
0,5000.0,36 months,10.65,162.87,10,RENT,24000.0,Verified,1,credit_card,27.65,0.0,1.0,3.0,0.0,13648.0,83.7,9.0
1,2500.0,60 months,15.27,59.83,0,RENT,30000.0,Source Verified,0,car,1.00,0.0,5.0,3.0,0.0,1687.0,9.4,4.0
2,2400.0,36 months,15.96,84.33,10,RENT,12252.0,Not Verified,1,small_business,8.72,0.0,2.0,2.0,0.0,2956.0,98.5,10.0
3,10000.0,36 months,13.49,339.31,10,RENT,49200.0,Source Verified,1,other,20.00,0.0,1.0,10.0,0.0,5598.0,21.0,37.0
4,5000.0,36 months,7.90,156.46,3,RENT,36000.0,Source Verified,1,wedding,11.20,0.0,3.0,9.0,0.0,7963.0,28.3,12.0


### Creating dummy variables
Categorical columns:
- `home_ownership`
- `verification_status`
- `purpose`
- `term`

In [20]:
dummy_df = pd.get_dummies(loans_2007[[
    'home_ownership', 'verification_status', 'purpose', 'term'
]])
loans_2007 = pd.concat([loans_2007,dummy_df],axis=1) #adding dummy cols to df

drop_cols = ['home_ownership', 'verification_status', 'purpose', 'term']
loans_2007.drop(drop_cols,axis=1, inplace=True) #drop text cols

## Prediction models

### Error metric

In [21]:
possibilities = {
    "loan_status": [0,0,1,1],
    "prediction": [1,0,1,0],
    "error": ["False Positive",
           "True Negative",
           "True Positive",
           "False Negative"]
}
print("Possible outcomes of a binary classification prediction:\n")
print(pd.DataFrame(possibilities).to_string(index=False))

Possible outcomes of a binary classification prediction:

 loan_status  prediction           error
           0           1  False Positive
           0           0   True Negative
           1           1   True Positive
           1           0  False Negative


__Loan status - possible outcomes__:
- 1: Loan will be paid off on time.
- 0: Loan will not be paid off on time.

__Prediction errors__:
- *False Positive*: Our model predicts 1 (will be paid off on time) but the actual status is 0 (not paid off on time). __This is our worst case scenario, since this approving such a loan would make us lose money.__

- *False Negative*: Our model predicts 0 (not paid off on time) but the actual status is 1 (will be paid off on time). 

__Class imbalance__:

Since we have way more loans that were paid off on time (1 values in our target `loans_2007['loan_status']`), our model will be more prone to predict that a loan will paid off on time. For this reason, using accuracy to measure the error metric is not the best approach.

To determine our error metric, we will look at following parameters:

__(rewrite formulas and explain)__
- *High recall/True positive rate*: $ tpr = tp / (tp + fn) $
- *Low fall-out/False positive rate*: $False_prate = false_p / (false_p + true_n)$

The higher our recall, the better our model will predict positive cases -we will grant more loans that will be paid off, and the lower our fall-out, the better our model will predict negative cases -we will reject more loans that would not be paid off. These two ratios are correlated and will fluctuate. 

Optimizing these ratios will yield the highest chances of gaining (or not losing) money when using our model to approve or reject a loan.

## Logistic regression

K-Fold cross validation.


In order to work around our class imbalance, we will penalize some rows by setting the `class_weight` parameter to `balanced` in our LogisticRegression instance. Our penalty will be inversely proportional to the class frequencies.

![Diagram of k-fold cross validation](https://upload.wikimedia.org/wikipedia/commons/thumb/b/b5/K-fold_cross_validation_EN.svg/521px-K-fold_cross_validation_EN.svg.png)
_Diagram of k-fold cross validation_

In [28]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import accuracy_score

features = loans_2007.drop(['loan_status'],axis=1) # drop target column
target = loans_2007["loan_status"]

# Create a LogisticRegression instance 
# balanced class_weight will penalize loan_status = 1 -higher frequency
lr = LogisticRegression(class_weight='balanced')

predictions = cross_val_predict(lr, features, target, cv=3)
predictions = pd.Series(predictions)

# True negatives
tn_bool_mask = loans_2007[(predictions == 0) & (loans_2007["loan_status"] == 0)]
tn = tn_bool_mask.shape[0]
# True positives
tp_bool_mask = loans_2007[(predictions == 1) & (loans_2007["loan_status"] == 1)]
tp = tp_bool_mask.shape[0]
# False negatives
fn_bool_mask = loans_2007[(predictions == 0) & (loans_2007["loan_status"] == 1)]
fn = fn_bool_mask.shape[0]
# False positives
fp_bool_mask = loans_2007[(predictions == 1) & (loans_2007["loan_status"] == 0)]
fp = fp_bool_mask.shape[0]

# Computing the true positive rate
tpr = tp/(tp+fn)

# Computing the false positive rate
fpr = fp/(fp + tn)

# Accuracy score - used only for model comparison purposes
accuracy = accuracy_score(loans_2007["loan_status"], predictions)

print("True positive rate: {:.2f}%".format(tpr*100))
print("False positive rate: {:.2f}%".format(fpr*100)) 
print("Number of false positives: {}".format(fp))
print("Accuracy score: {:.2f}%".format(accuracy*100))

True positive rate: 67.02%
False positive rate: 38.86%
Number of false positives: 2094
Accuracy score: 66.18%


## Random forest classifier model

In [27]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(random_state=1,class_weight='balanced')

predictions = cross_val_predict(rfc, features, target, cv=3)
predictions = pd.Series(predictions)

# True negatives
tn_bool_mask = loans_2007[(predictions == 0) & (loans_2007["loan_status"] == 0)]
tn = tn_bool_mask.shape[0]
# True positives
tp_bool_mask = loans_2007[(predictions == 1) & (loans_2007["loan_status"] == 1)]
tp = tp_bool_mask.shape[0]
# False negatives
fn_bool_mask = loans_2007[(predictions == 0) & (loans_2007["loan_status"] == 1)]
fn = fn_bool_mask.shape[0]
# False positives
fp_bool_mask = loans_2007[(predictions == 1) & (loans_2007["loan_status"] == 0)]
fp = fp_bool_mask.shape[0]

# Computing the true positive rate
tpr = tp/(tp+fn)

# Computing the false positive rate
fpr = fp/(fp + tn)

# Accuracy score - used only for model comparison purposes
accuracy = accuracy_score(loans_2007["loan_status"], predictions)

print("True positive rate: {:.2f}%".format(tpr*100))
print("False positive rate: {:.2f}%".format(fpr*100)) 
print("Number of false positives: {}".format(fp))
print("Accuracy score: {:.2f}%".format(accuracy*100))

True positive rate: 97.01%
False positive rate: 91.82%
Number of false positives: 4948
Accuracy score: 84.30%


## Conclusion: Logistic regression VS Random forest classifier
As we can see, our false positive